# Chat Bot
***
## Table of Contents
***

## 1. Introduction


## 2. Environmental Variables
The API keys and environmental variables should never be hardcoded or exposed publicly. The [python-dotenv](https://pypi.org/project/python-dotenv/) library facilitates secure access to variables defined in a `.env` file.

In [1]:
import os

try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    raise ImportError("Error: 'python-dotenv not installed")

print(f"Project name: {os.environ['LANGSMITH_PROJECT']}")

Project name: lc_fundamentals


## 3. Loading Model

In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage

model_name = "gemini-2.5-flash"
provider = "google_genai"
model = init_chat_model(model=model_name, model_provider=provider)

## 4. Conversational Memory

In [3]:
model.invoke(input="Hi, I am John.")

AIMessage(content="Hello John! Nice to meet you. I'm an AI, a large language model.\n\nHow can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f21f8fcd-46b3-4a2e-adcb-593ced29cecd-0', usage_metadata={'input_tokens': 7, 'output_tokens': 562, 'total_tokens': 569, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 535}})

In [4]:
model.invoke(input="What is my name?")

AIMessage(content="As an AI, I don't have access to personal information about you, including your name. I don't retain memory of past conversations or user identities.\n\nIf you'd like me to know your name for our current chat, please feel free to tell me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9a1d7d10-69ae-493d-9a2a-5e3aca9dc9c6-0', usage_metadata={'input_tokens': 6, 'output_tokens': 446, 'total_tokens': 452, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 390}})

In [5]:
model.invoke(
    [
        HumanMessage(content="Hi, I am John."),
        AIMessage(content="Hello John, how can I help you today?"),
        HumanMessage(content="What is my name?"),
    ]
)

AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--efc4124f-39bc-4f93-9570-61fb41536698-0', usage_metadata={'input_tokens': 24, 'output_tokens': 42, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 37}})

### Traditional Methods

In previous versions of LangChain (prior to 0.3), memory classes were used to store and manage conversation history internally within each chain or memory object. The following classes are **deprecated** as of August 2025 (version $\geq$ 0.3), but I include them here for learning purposes:

- **ConversationBufferMemory**: Stores the entire conversation history. This is the simplest method for managing memory.
- **ConversationBufferWindowMemory**: Retains only the last $k$ messages in the conversation.
- **ConversationSummaryMemory**: Rather than storing the entire history, this class generates and retains a summary of the conversation.
- **ConversationSummaryBufferMemory**: Summarises each exchange and retains only the most recent $k$ summaries.

These classes were later used in combination with the `RunnableWithMessageHistory` class, which wraps both the chain and the memory implementation. As of LangChain version 0.3, although `RunnableWithMessageHistory` is not explicitly deprecated, it is strongly recommended to migrate to LangGraph persistence, which offers a more robust and scalable approach to managing message history.

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/var/folders/71/96lm97d97jd1rltw9bhztxc80000gn/T/ipykernel_58507/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [7]:
memory.chat_memory.add_user_message(message="Hi, I am John.")
memory.chat_memory.add_ai_message(message="Hello John, how can I assist you today?")
memory.chat_memory.add_user_message(message="Who won the Champions League in 2024?")
memory.chat_memory.add_ai_message(
    message="In 2024, Manchester City won the UEFA Champions League."
)

memory.load_memory_variables(
    {}
)  # Need to load variables for memory type - none in this example

{'history': [HumanMessage(content='Hi, I am John.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John, how can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won the Champions League in 2024?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='In 2024, Manchester City won the UEFA Champions League.', additional_kwargs={}, response_metadata={})]}

In [8]:
from langchain.chains import ConversationChain

chain = ConversationChain(llm=model, memory=memory, verbose=True)

chain.invoke({"input": "Remind me what we spoke about earlier."})

/var/folders/71/96lm97d97jd1rltw9bhztxc80000gn/T/ipykernel_58507/4191846825.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(llm=model, memory=memory, verbose=True)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, I am John.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello John, how can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who won the Champions League in 2024?', additional_kwargs={}, response_metadata={}), AIMessage(content='In 2024, Manchester City won the UEFA Champions League.', additional_kwargs={}, response_metadata={})]
Human: Remind me what we spoke about earlier.
AI:

> Finished chain.


{'input': 'Remind me what we spoke about earlier.',
 'history': [HumanMessage(content='Hi, I am John.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John, how can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won the Champions League in 2024?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='In 2024, Manchester City won the UEFA Champions League.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Remind me what we spoke about earlier.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Certainly, John! We've had a lovely chat so far.\n\nJust a moment ago, you asked me about the winner of the **Champions League in 2024**. And I informed you that **Manchester City** were the champions, lifting that prestigious UEFA Champions League trophy!\n\nBefore that, you introduced yourself to me as **John**, and I, of course, greeted you and asked how I could assist you 

### Modern Methods


- **MemorySaver**: An in-memory checkpoint class for saving the state (chat messages) during execution. It's suitable for prototypes; for production, it should be swapped with persistant backends (e.g., SQLite, Postgres).
- **StateGraph + MessagesState**: LangGraph's state machine and schema for storing the message state. `START` is a special entry node.

In [9]:
from langchain_core.messages.base import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

StateGraph is a core graph structure in LangGraph. It has to be initialised with the schema that describes the states to keep (in this case, conversation messages).

In [10]:
workflow = StateGraph(state_schema=MessagesState)

The following functions accepts the state (including the message history). It passes the messages to the model, receives the response, and returns the new state including updates messages.

In [11]:
def call_model(state: MessagesState) -> dict[str, BaseMessage]:
    response = model.invoke(state["messages"])
    return {"messages": response}

- `add_edge`: Defines the flow; after `START`, the workflow progresses to the `model` node.
- `add_node`: Registers the `model` node and links it to the `call_model()` function.

In [12]:
workflow.add_edge(start_key=START, end_key="model")
workflow.add_node(node="model", action=call_model)

After setting edges and nodes, we add memory and compile the app. 

In [13]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

The `config` dictionary can contain a `configurable` key, which holds runtime parameters for LangGraph's execution. The `thread_id` value within this key identifies each conversation. If multiple usrs are chatting, each one can be assigned a unique `thread_id` (typically their user ID).

In [14]:
config = {"configurable": {"thread_id": "user_id_123"}}

Finally, the `app.invoke()` runs the LangGraph workflow using the given initial state. The `config` parameter ensures the conversation updates and is checkpointed under the specified thread ID.

In [15]:
query = "Hi, I am John."
input_messages = [HumanMessage(query)]
output = app.invoke(input={"messages": input_messages}, config=config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello John! Nice to meet you. I'm an AI assistant. How can I help you today?


## 5. Prompt Templates
A prompt template provides a structured way of creating inputs for language models where parts of the prompt can be dynamically changed based on context or user input.

Prompts in LangChain can be split into three components:
- **System Prompt**: Gives instructions or a personality to the LLM model. This prompt determines the behaviour or characteristics of the model.
- **User Prompt**: Input given by a user.
- **AI Prompt**: Output generated by the model.

In [16]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

prompt_template = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="You are an AI receptionist at a five-star hotel. Welcome and helps guests to the best of your ability in {language}.",
            input_variables=["language"],
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Earlier, `MessageState` contained only a single field (`messages`). To store multiple state variables, such as language, ID, or any custom matadata, we need to define a new `State` class explicitly with all required fields. It allows the graph to manage, validate, and pass around all relevant state components clearly while reducing bugs by enabling type checking.

In [ ]:
from typing import Sequence
from langchain_core.messages.base import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):  # Custom State class
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)  # Pass it to StateGraph


def call_model(state: State) -> dict[str, BaseMessage]:
    prompt = prompt_template.invoke(input=state)
    response = model.invoke(input=prompt)
    return {"messages": [response]}


workflow.add_edge(start_key=START, end_key="model")
workflow.add_node(node="model", action=call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "A11111"}}
query = "I want to make a reservation"
language = "French"
input_messages = [HumanMessage(query)]
output = app.invoke(
    input={"messages": input_messages, "language": language}, config=config
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bonjour et bienvenue à notre hôtel. Je suis votre réceptionniste AI.

Je serais ravie de vous aider à faire une réservation. Pourriez-vous me dire, s'il vous plaît :

1.  **Quelles sont les dates** d'arrivée et de départ souhaitées ?
2.  **Combien de personnes** séjourneront (adultes et enfants, si applicable) ?
3.  **Quel type de chambre** vous intéresserait (par exemple, chambre standard, suite, chambre avec vue sur la mer) ?

Une fois que j'aurai ces informations, je pourrai vérifier la disponibilité et les options pour vous.
